# AABC Cumulative Recruitment Statistics and Visuals


# DO NOT PUBLISH THESE NUMBERS 
### These numbers are 'live' as of the timestamp in their filename.  As such, they are subject to updates as discrepancies are resolved between local and central reporting.  



In [ ]:
# check out  ~/cron/aabc_recruits.sh
#

In [ ]:
#load some libraries
import pandas as pd
import seaborn as sns
import requests
from ccf.box import LifespanBox
import yaml
from functions import *
from config import *
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import warnings
from itables import show
import itables.options as opt
#warnings.filterwarnings('ignore')

In [ ]:
opt.maxBytes = "512KB"

In [ ]:
print(date.today().strftime("%m/%d/%Y"))

In [ ]:
#load HCA inventory 
config = LoadSettings()
secret=pd.read_csv(config['config_files']['secrets'])
box = LifespanBox(cache="./tmp")
pathp=box.downloadFile(config['hcainventory'])
ids=pd.read_csv(pathp)
intradb=pd.read_csv(config['config_files']['PCP'])


In [ ]:
#jdump=curl -X GET "https://intradb.humanconnectome.org/data/archive/projects/AABC_UMN_ITK/experiments?format=json"    -u intradb.user[0]:intradb.password[0]      -H "Content-Type: application/x-www-form-urlencoded"
url = "https://intradb.humanconnectome.org/data/archive/projects/AABC_UMN_ITK/experiments?format=json"
url2="https://intradb.humanconnectome.org/data/archive/projects/AABC_MGH_ITK/experiments?format=json"
response = requests.get(url, auth=(intradb.user[0],intradb.auth[0]), headers={
    "Content-Type": "application/x-www-form-urlencoded"
})
response2 = requests.get(url2, auth=(intradb.user[0],intradb.auth[0]), headers={
    "Content-Type": "application/x-www-form-urlencoded"
})

response.raise_for_status()
data = response.json()
#print(data.keys())  # Temporarily for debugging
experiments = data.get('ResultSet', {}).get('Result', [])
df = pd.DataFrame(experiments)
#df.head()

response2.raise_for_status()
data2 = response2.json()
#print(data2.keys())  # Temporarily for debugging
experiments2 = data2.get('ResultSet', {}).get('Result', [])
df2 = pd.DataFrame(experiments2)
#print(df2.head())


In [ ]:
#find the 7T folks
DF=pd.concat([df,df2],axis=0)
DF.head()
subjects7T=DF.loc[DF.label.str.contains('7T')][['label']]
split_cols = subjects7T['label'].str.split('_', expand=True)
subjects7T['subject'] = split_cols[0]
subjects7T['redcap_event'] = split_cols[1]

subjects7T=subjects7T[['subject','redcap_event']]     
subjects7T.head()
subjects7T=subjects7T.drop_duplicates()

In [ ]:
### CHECK WITH ANGELA ##########
### DONT FORGET TO DO SOMETHING ABOUT SUBJECT WE SHOULD DELETE
### DONT WANT TO ACCIDENTALLY DROP WITHDRAWNS

DNR = ["HCA7787304_V1", "HCA6276071_V1", "HCA6229365_V1", "HCA9191078_V1", "HCA6863086_V1"]
#These guys accidentally recruited as V2
v2oops=['HCA6686191','HCA7296183']

In [ ]:
hcaids=ids.subject.drop_duplicates()
#for later use in getting the last visit for each participant in HCA so that you can later make sure that person is starting subsequent visit and not accidentally enrolled in the wrong arm
hca_lastvisits=ids[['subject','redcap_event']].loc[ids.redcap_event.isin(['V1','V2'])].sort_values('redcap_event').drop_duplicates(subset='subject',keep='last')

In [ ]:
#load AABC report
aabcarms = redjson(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0])
hcpa = redjson(tok=secret.loc[secret.source=='hcpa','api_key'].reset_index().drop(columns='index').api_key[0])
#just a report
aabcreport = redreport(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0],reportid='51031')


In [ ]:
#download the inventory report from AABC for comparison
aabcinvent=getframe(struct=aabcreport,api_url=config['Redcap']['api_url10']).drop(columns='dob')

In [ ]:
## CODE RED!

In [ ]:
#find subjectts who have completed a visit and are not in DNR list - save for later
reds=aabcinvent.loc[aabcinvent.register_visit_complete =='2'][['study_id']]
inperson=list(reds.study_id.unique())
reds2=aabcinvent.loc[(aabcinvent.study_id.isin(inperson)) & (~(aabcinvent.subject_id =='')) & (~(aabcinvent.subject_id.isin(DNR)))]
inpersonHCAid=list(reds2.subject_id.unique())

In [ ]:
##this is too complicated
#study_id=config['Redcap']['datasources']['aabcarms']['redcapidvar']
#study_id


In [ ]:

#slim selects just the registration event (V0) because thats where the ids and legacy information is kept.
slim=aabcinvent[['study_id','redcap_event_name','subject_id','legacy_yn','site','v0_date']].loc[(aabcinvent.redcap_event_name.str.contains('register'))]

#compare aabc ids against hcaids and whether legacy information is properly accounted for (e.g. legacy variable flags and actual event in which participannt has been enrolled.
fortest=pd.merge(hcaids,slim,left_on='subject',right_on='subject_id',how="outer",indicator=True)
#fortest._merge.value_counts()
legacyarms=['register_arm_1','register_arm_2','register_arm_3','register_arm_4','register_arm_5','register_arm_6','register_arm_7','register_arm_8']

In [ ]:
# First batch of flags: Look for legacy IDs that don't actually exist in HCA
ft=fortest.loc[(fortest._merge=='right_only') & ((fortest.legacy_yn=='1')|(fortest.redcap_event_name.isin(legacyarms)))]
#remove the TEST subjects -- probably better to do this first, but sigh.
ft=ft.loc[~((ft['subject_id']=='')|(ft['subject_id'].str.upper().str.contains('TEST')))]
qlist1=pd.DataFrame()
if not ft.empty:
    ft['reason']='Subject found in AABC REDCap Database with legacy indications whose ID was not found in HCP-A list'
    ft['issueCode']='AE1001'
    ft['datatype']='REDCap'
    ft['code']='RED'
    qlist1=ft[['study_id','subject_id', 'redcap_event_name', 'site','reason','code','v0_date','datatype']]
    for s in list(ft['subject_id'].unique()):
        print('CODE RED :',s,': Subject found in AABC REDCap Database with legacy indications whose ID was not found in HCP-A list')

#2nd batch of flags: if legacy v1 and enrolled as if v3 or v4 or legacy v2 and enrolled v4
ft2=fortest.loc[(fortest._merge=='both') & ((fortest.legacy_yn != '1')|(~(fortest.redcap_event_name.isin(legacyarms))))]
qlist2=pd.DataFrame()
if not ft2.empty:
    ft2['reason']='Subject found in AABC REDCap Database with legacy indications whose ID was not found in HCP-A list'
    ft2['code']='RED'
    ft2['issueCode'] = 'AE1001'
    ft2['datatype']='REDCap'
    qlist2 = ft2[['study_id','subject_id', 'redcap_event_name', 'site','reason','code','v0_date','datatype']]
    for s2 in list(ft2['subject_id'].unique()):
        print('CODE RED :',s2,': Subject found in AABC REDCap Database with an ID from HCP-A study but no legacyYN not checked')



In [ ]:
#THIS IS NOT DOING WHAT IT SAYS ITS DOING
# Check if subject fail the screen but came in for visit
#PASSEDSCREEN:  1, Yes | 0, No | 2, Hold - pending Cohort C

#PRESTON'S DEFINITION 
pass_failed=aabcinvent.loc[(aabcinvent.passedscreen =='2') & (aabcinvent['subject_id'].astype(str).str.strip() != '')][['subject_id', 'study_id', 'redcap_event_name', 'site','v0_date']]
qlist4=pd.DataFrame()
if not pass_failed.empty:
    pass_failed['reason']='subject did not pass screen but came in for imaging - need confirmation'
    pass_failed['code']='RED'
    pass_failed['issueCode'] = 'AE1001'
    pass_failed['datatype']='REDCap'
    qlist4 = pass_failed[['subject_id', 'study_id', 'redcap_event_name', 'site','reason','code','v0_date','datatype']]
    for s4 in list(pass_failed[study_id].unique()):
        print('CODE RED :',s4,': subject did not pass screen but came in for imaging - need confirmation')

In [ ]:
#if legacy v1 and enrolled as if v3 or v4 or legacy v2 and enrolled v4
#get last visit
hca_lastvisits["next_visit"]=''
#idvisits rolls out the subject ids to all visits. get subects current visit for comparison with last visit
aabcidvisits=idvisits(aabcinvent,keepsies=['study_id','redcap_event_name','site','subject_id','v0_date','event_date'])
sortaabc=aabcidvisits.sort_values(['study_id','redcap_event_name'])
sortaabcv=sortaabc.loc[~(sortaabc.redcap_event_name.str.contains('register'))]
sortaabcv.drop_duplicates(subset=['study_id'],keep='first')
#print("OOOPSs:",sortaabcv.loc[sortaabcv.subject.isin(v2oops)])
#add 1 to last visit from HCA
#also set up for checking to make sure not initiating same visit
hca_lastvisits.next_visit=hca_lastvisits.redcap_event.str.replace('V','').astype('int') +1
hca_lastvisits["next_visit2"]="V"+hca_lastvisits.next_visit.astype(str)
hca_lastvisits2=hca_lastvisits.drop(columns=['redcap_event','next_visit'])

In [ ]:
#check that current visit in AABC is the last visit in HCA + 1
check=pd.merge(hca_lastvisits2,sortaabcv,left_on=['subject','next_visit2'],right_on=['subject','redcap_event'],how='outer',indicator=True)
check=check.loc[check._merge !='left_only']
wrongvisit=check.loc[check._merge=='right_only']
wrongvisit=wrongvisit.loc[~(wrongvisit.redcap_event.isin(['AP']))]#,'v1_inperson_arm_10','v1_inperson_arm_12']))]
wrongvisit=wrongvisit.loc[wrongvisit.next_visit2.isnull()==False]

In [ ]:
qlist3=pd.DataFrame()
if not wrongvisit.empty:
    wrongvisit['reason']='Subject found in AABC REDCap Database initiating the wrong visit sequence (e.g. V3 insteady of V2'
    wrongvisit['code']='RED'
    wrongvisit['issueCode'] = 'AE1001'
    wrongvisit['datatype']='REDCap'
    qlist3 = wrongvisit[['subject', 'study_id', 'redcap_event_name', 'site','reason','code','v0_date','event_date','datatype']]
    qlist3=qlist3.rename(columns={'subject':'subject_id'})
    for s3 in list(wrongvisit['subject'].unique()):
        if s3 !='':
            print('CODE RED (if HCA6911778 ignore) :',s3,': Subject found in AABC REDCap Database initiating the wrong visit sequence (e.g. V3 insteady of V2')
            qlist3=qlist3.loc[~(qlist3.subject_id=='HCA6911778')].copy()

In [ ]:
#check to make sure they aren't initiating the same visit.
check2=pd.merge(hca_lastvisits[['subject','redcap_event']],sortaabcv,left_on=['subject','redcap_event'],right_on=['subject','redcap_event'],how='inner')
check2=check2.loc[~(check2.subject.isin(v2oops))]
qlist32=pd.DataFrame()
if not check2.empty:
    check2['reason']='Subject found in AABC REDCap Database initiating the wrong visit sequence (e.g. V3 insteady of V2'
    check2['code']='RED'
    check2['issueCode'] = 'AE1001'
    check2['datatype']='REDCap'
    qlist32 = check2[['subject', 'study_id', 'redcap_event_name', 'site','reason','code','v0_date','event_date','datatype']]
    qlist32=qlist32.rename(columns={'subject':'subject_id'})
    for s3 in list(check2['subject'].unique()):
        if s3 !='':
            print('CODE RED (if HCA6911778 ignore) :',s3,': Subject found in AABC REDCap Database initiating the wrong visit sequence (e.g. V3 insteady of V2')
            qlist32=qlist32.loc[~(qlist32.subject_id=='HCA6911778')].copy()

In [ ]:
#test subjects that need to be deleted
tests=aabcinvent.loc[(aabcinvent['subject_id'].str.upper().str.contains('TEST')) | (aabcinvent['subject_id'].str.upper().str.contains('PRAC')) | (aabcinvent['subject_id'].str.upper().str.contains('DEMO'))][['study_id','subject_id','redcap_event_name']]
qlist5=pd.DataFrame()
if not tests.empty:
    tests['reason']='HOUSEKEEPING : Please delete test subject.  Use test database when practicing'
    tests['code']='HOUSEKEEPING'
    tests['datatype']='REDCap'
    tests['issueCode'] = 'AE6001'
    qlist5 = tests[['subject_id', 'study_id', 'redcap_event_name', 'site','reason','code','v0_date','event_date','datatype']]
    for s5 in list(tests['subject_id'].unique()):
        print('HOUSEKEEPING : Please delete test subject:', s5)

In [ ]:
## End of CODE RED

In [ ]:

#download the inventory report from AABC for comparison
print("Connecting to REDCap")
aabcinvent=getframe(struct=aabcreport,api_url=config['Redcap']['api_url10']).drop(columns='dob')


In [ ]:
#aabcinvent.shape

In [ ]:
### PPPPP
#CLEAN UP OPTION FOR PRODUCING PLOTS BY VISIT instead of SUBJECT


#api metadata
#aabcarms = redjson(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0])
#hcpa = redjson(tok=secret.loc[secret.source=='hcpa','api_key'].reset_index().drop(columns='index').api_key[0])

#report deets
#aabcreport = redreport(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0],reportid='51031')

#download the inventory report from AABC
#aabcinvent=getframe(struct=aabcreport,api_url=config['Redcap']['api_url10']).drop(columns='dob')
#print("shape of inventory before filters or merges",aabcinvent.shape)


aabcinventbak=aabcinvent.copy()

In [ ]:
#aabcinventbak.redcap_event_name.value_counts()

In [ ]:
aabcinvent=aabcinventbak.copy()
rollupvars=['subject_id','site','sex','ethnic','racial','croms_income','counterbalance_1st', 'passedscreen']
rolldownvars=['withdrawal','withdrawal_pt_reason','withdraw_ptsite','withdraw_other','date_last_visit','date_pmnotify',
'croms_stopinterven','croms_intervenstopreason','croms_stopstudy','croms_stopstudyreason']
#roll up variables that only exist in registration so that you can produce stats by visits
#righ to filter on empty study id or should we be filtering on registration?
aabcinvent=aabcinvent.drop(columns=rollupvars).merge(aabcinvent.loc[aabcinvent['redcap_event_name'].str.contains('register')][['study_id']+rollupvars],on='study_id',how='left')
aabcinvent=aabcinvent.drop(columns=rolldownvars).merge(aabcinvent.loc[aabcinvent['redcap_event_name'].str.contains('end_of_study')][['study_id']+rolldownvars],on='study_id',how='left')
aabcenroll=aabcinvent.loc[(aabcinvent.redcap_event_name.str.contains('inperson'))].drop_duplicates(subset='study_id',keep='first')#.register_visit_complete.value_counts()
#aabcenroll.register_visit_complete.value_counts()
#aabcenroll.loc[~(aabcenroll.register_visit_complete=='0')].passedscreen.value_counts()


#aabcenroll.loc[aabcenroll.passedscreen==''][['study_id','redcap_event_name','event_date','subject_id','site']]#,'register_visit_complete','passedscreen']]
#aabcinvent=aabcinvent.drop(columns=rollupvars).merge(aabcinvent.loc[~(aabcinvent.subject_id =='')][['study_id']+rollupvars],on='study_id',how='left')
#print(aabcinvent.shape)

#print("double check shape of inventory before filters:",aabcinvent.shape)
#print("number of unique REDCap IDs:",str(len(aabcinvent.study_id.unique())))
#print("number of unique subjects before filters:",str(len(aabcinvent.loc[~(aabcinvent.subject_id.str.strip()=='')].subject_id.unique())))



In [ ]:
#passed screen 
aabcinventpass = aabcinvent.loc[aabcinvent.passedscreen.isin(['1','2'])]# == '1'] 
#print("shape of inventory passed screen",aabcinventpass.shape)
print("shape of regsitrations passed screen",aabcinventpass.loc[aabcinventpass['redcap_event_name'].str.contains('register')].shape)

In [ ]:
aabcinventpass.loc[(aabcinventpass.redcap_event_name.str.contains('inperson'))].register_visit_complete.value_counts()

aabcinventpass.loc[(aabcinventpass.passedscreen=='1') & (aabcinventpass.redcap_event_name.str.contains('inperson'))]
x=aabcinventpass.loc[(aabcinventpass.passedscreen=='1') & (aabcinventpass.redcap_event_name.str.contains('inperson'))]
y=x.drop_duplicates(subset='study_id',keep='first').copy()
#x.head()
#list(x.columns)
#print(x.shape)
#aabcinventpass.loc[(aabcinventpass.passedscreen!='1') & (aabcinventpass.event_date.isnull()==False)]

In [ ]:

## Define the date range
#start_date = '2023-09-01'
##end_date = '2025-01-15'
#print(x.shape)
## Filter rows with event_date in the specified range
#filtered_x = x[(x['event_date'] >= start_date) & (x['event_date'] <= end_date)]
#filtered_x=filtered_x[['subject_id','event_date','redcap_event','Site']].sort_values(['subject_id','event_date']).copy()
#print(filtered_x.shape)
#filtered_x.redcap_event.value_counts()
#pd.crosstab(filtered_x.redcap_event,filtered_x.Site,margins=True)
## Count rows for each subject_id
##result_df = filtered_x.groupby('subject_id').size().reset_index(name='count')
##print(result_df.shape)
##result_df.columns=['subject_id','visits']
##result_df.head()
##result_df.visits.value_counts()

In [ ]:
#aabcinventpass.loc[(aabcinventpass.register_visit_complete=='2')].drop_duplicates(subset='study_id')# & (aabcinventpass.redcap_event_name.str.contains('inperson'))].drop_duplicates(subset='study_id')

In [ ]:
aabcinvent=aabcinventpass.copy()
aabcinvent['todaydate']=date.today()
#BAD formula:
#aabcinvent['dayspassed']=(pd.to_datetime(aabcinvent.todaydate) - pd.to_datetime(aabcinvent.event_date)).dt.days

#GOOD FORMULA:
aabcinvent['dayspassed']=(pd.to_datetime(aabcinvent.event_date)- pd.to_datetime('05/22/2022')).dt.days
aabcinvent=aabcinvent.sort_values('dayspassed')
#aabcinvent.to_csv('test.csv')
#print('data shape: ', str(aabcinvent.shape))
##print('selecting visits with completed registration (indicates they actually came in)')
#accrued=aabcinvent.loc[(aabcinvent.register_visit_complete=='2') & (aabcinvent.redcap_event_name.str.contains('inperson'))]
accrued = aabcinvent.loc[aabcinvent.redcap_event_name.str.contains('inperson')].drop_duplicates(subset='subject_id', keep='first')

#print('select first in-person visit: ', str(accrued.shape))
accrued = accrued.loc[accrued.register_visit_complete == '2'] 
#print('select visits with completed registration: ', str(accrued.shape))
#print('now the shape of the data is:',str(accrued.shape))

#uncomment if you want to get the event counts
#print(accrued.redcap_event_name.value_counts())
#accrued.loc[accrued.redcap_event_name.str.contains('v4')]

In [ ]:
print("Total number of unique subjects now:",str(len(accrued.subject_id.unique())))

In [ ]:
#THIS NO LONGER Necessary, but keeping just in case:
#DNR = ["HCA7787304_V1", "HCA6276071_V1", "HCA6229365_V1", "HCA9191078_V1", "HCA6863086_V1"]
##These guys accidentally recruited as V2
#v2oops=['HCA6686191','HCA7296183']

#droplist=['HCA7787304','HCA7142156','HCA6863086','HCA6276071','HCA6229365','HCA9191078']
#droplist=['HCA7787304','HCA7142156','HCA6863086','HCA6276071','HCA6229365','HCA9191078', 'HCA8743995','HCA62276071',
#          'HCA6418974', 'HCA9841899', 'HCA6298788', 'HCA9515381']
#accruedclean=accrued.loc[~(accrued.subject_id.isin(droplist))]
#accruedclean=accruedclean.loc[~(accruedclean.passedscreen.isna())]
#print('drop any stragglers that failed screening or withdrew')
#print(accruedclean.shape)

#select subjects visit before Oct 1
#accruedclean = accruedclean[accruedclean['event_date'] < '2024-10-01']
#print(accruedclean.shape)

#forplot=accruedclean.copy()
forplot=accrued.copy()
#use age at visit (age_visit) instead of age at baseline (age).
forplot=forplot.drop(columns=['age']).rename(columns={'age_visit':'age'})
#print(list(forplot.columns))


In [ ]:
print(forplot.shape)
print("drop the grifter")
forplot=forplot.loc[~(forplot.subject_id=='HCA6539885')]
forplot.shape

In [ ]:
forplot.event_date
#jan14 = forplot.loc[forplot.event_date < '2025-01-15']['subject_id'].unique()
#jan15 = forplot.loc[forplot.event_date <= '2025-01-15']['subject_id'].unique()
#diff = [i for i in jan15 if i not in jan14]
#diff
#'HCA6539885 is grifter '
may2025 =forplot.loc[(forplot.event_date <= '2025-04-30') | (~(forplot.subject_id=='HCA6539885'))]
print(len(may2025.subject_id.unique()))
apr2025 =forplot.loc[(forplot.event_date <= '2025-03-31') & (~(forplot.subject_id=='HCA6539885'))]
print(len(apr2025.subject_id.unique()))

In [ ]:
#PREPARE DATA FOR CUMULATIVE PLOTS  
S=pd.get_dummies(forplot.sex, prefix='sex')

#forplot['sexsum']=pd.to_numeric(forplot.sex, errors='coerce').cumsum()
forplot['malesum']=pd.to_numeric(S.sex_1, errors='coerce').cumsum()
forplot['femalesum']=pd.to_numeric(S.sex_2, errors='coerce').cumsum()
forplot['Sex']=forplot.sex.replace({'1':'Male','2':'Female'})

S0=pd.get_dummies(forplot.counterbalance_1st, prefix='CB')
forplot['CB3sum']=pd.to_numeric(S0.CB_3, errors='coerce').cumsum()
forplot['CB4sum']=pd.to_numeric(S0.CB_4, errors='coerce').cumsum()
forplot['Counterbalance']=forplot.counterbalance_1st.replace({'3':'CB3','4':'CB4'})

#forplot[['subject_id','dayspassed','malesum','femalesum','Sex']].head(20)

In [ ]:
#forplot.redcap_event_name.value_counts()
forplot['Cohort']=''
forplot.loc[(forplot.redcap_event_name.str.contains("arm_1")) | (forplot.redcap_event_name.str.contains("arm_2")) |(forplot.redcap_event_name.str.contains("arm_3")) |(forplot.redcap_event_name.str.contains("arm_4")) ,'Cohort']='A'
forplot.loc[(forplot.redcap_event_name.str.contains("arm_5")) | (forplot.redcap_event_name.str.contains("arm_6")) |(forplot.redcap_event_name.str.contains("arm_7")) |(forplot.redcap_event_name.str.contains("arm_8")) ,'Cohort']='B'
forplot.loc[(forplot.redcap_event_name.str.contains("arm_9")) | (forplot.redcap_event_name.str.contains("arm_10")) |(forplot.redcap_event_name.str.contains("arm_11")) |(forplot.redcap_event_name.str.contains("arm_12")) ,'Cohort']='New'
#forplot.Cohort

In [ ]:
forplot.subject_id
forplotbak=forplot.copy()

In [ ]:
#1, Native American/Alaskan Native | 2, Asian | 3, Black or African American | 4, Native Hawaiian or Other Pacific Is | 5, White | 6, More than one race | 99, Unknown or Not reported
S2=pd.get_dummies(forplot.racial, prefix='race')
#print(S2.head())
forplot['whitesum']=pd.to_numeric(S2.race_5, errors='coerce').cumsum()
#forplot['natpacsum']=pd.to_numeric(S2.race_4, errors='coerce').cumsum()
forplot['blacksum']=pd.to_numeric(S2.race_3, errors='coerce').cumsum()
forplot['asiansum']=pd.to_numeric(S2.race_2, errors='coerce').cumsum()
forplot['natamersum']=pd.to_numeric(S2.race_1, errors='coerce').cumsum()
forplot['moret1sum']=pd.to_numeric(S2.race_6, errors='coerce').cumsum()
forplot['nasum']=pd.to_numeric(S2.race_99, errors='coerce').cumsum()
forplot['Race']=forplot.racial.replace({'1':'Nat Amer/Alaskan','2':'Asian','3':'Black','4':'Nat Hawaiian/PI','5':'White','6':'More than one','99':'Unknown'})

#thnicity
S3=pd.get_dummies(forplot.ethnic, prefix='ethnicity')
forplot['hispanicsum']=pd.to_numeric(S3.ethnicity_1, errors='coerce').cumsum()
forplot['nonhispanicsum']=pd.to_numeric(S3.ethnicity_2, errors='coerce').cumsum()
forplot['unkhispsum']=pd.to_numeric(S3.ethnicity_3, errors='coerce').cumsum()
forplot['Ethnicity']=forplot.ethnic.replace({'1':'Hispanic','2':'Non-Hispanic','3':'Unknown'})

#sites
S4=pd.get_dummies(forplot.site, prefix='site')
forplot['wusum']=pd.to_numeric(S4.site_4, errors='coerce').cumsum()
forplot['umnsum']=pd.to_numeric(S4.site_3, errors='coerce').cumsum()
forplot['mghsum']=pd.to_numeric(S4.site_1, errors='coerce').cumsum()
forplot['uclasum']=pd.to_numeric(S4.site_2, errors='coerce').cumsum()
forplot['Site']=forplot.site.replace({'1':'MGH','2':'UCLA','3':'UMN','4':'WashU'})



In [ ]:
                               
##ages
bins= [30,40,50,60,70,80,90,125]

forplot['ages']=pd.to_numeric(forplot.age)
forplot['AgeGroup'] = pd.cut(forplot['ages'], bins=bins,right=False)# labels=labels,
S5=pd.get_dummies(forplot.AgeGroup, prefix='age')

forplot['age30sum']=pd.to_numeric(S5['age_[30, 40)'], errors='coerce').cumsum()
forplot['age40sum']=pd.to_numeric(S5['age_[40, 50)'], errors='coerce').cumsum()
forplot['age50sum']=pd.to_numeric(S5['age_[50, 60)'], errors='coerce').cumsum()
forplot['age60sum']=pd.to_numeric(S5['age_[60, 70)'], errors='coerce').cumsum()
forplot['age70sum']=pd.to_numeric(S5['age_[70, 80)'], errors='coerce').cumsum()
forplot['age80sum']=pd.to_numeric(S5['age_[80, 90)'], errors='coerce').cumsum()
forplot['age90sum']=pd.to_numeric(S5['age_[90, 125)'],errors='coerce').cumsum()

In [ ]:
bins5= [35,40,45,50,55,60,65,70,75,80,85,90,125]

forplot['AgeGroup5'] = pd.cut(forplot['ages'], bins=bins5,right=False)# labels=labels,
S55=pd.get_dummies(forplot.AgeGroup5, prefix='age5')

forplot['age35sum5']=pd.to_numeric(S55['age5_[35, 40)'], errors='coerce').cumsum()
forplot['age40sum5']=pd.to_numeric(S55['age5_[40, 45)'], errors='coerce').cumsum()
forplot['age45sum5']=pd.to_numeric(S55['age5_[45, 50)'], errors='coerce').cumsum()
forplot['age50sum5']=pd.to_numeric(S55['age5_[50, 55)'], errors='coerce').cumsum()
forplot['age55sum5']=pd.to_numeric(S55['age5_[55, 60)'], errors='coerce').cumsum()
forplot['age60sum5']=pd.to_numeric(S55['age5_[60, 65)'], errors='coerce').cumsum()
forplot['age65sum5']=pd.to_numeric(S55['age5_[65, 70)'], errors='coerce').cumsum()
forplot['age70sum5']=pd.to_numeric(S55['age5_[70, 75)'], errors='coerce').cumsum()
forplot['age75sum5']=pd.to_numeric(S55['age5_[75, 80)'],errors='coerce').cumsum()
forplot['age80sum5']=pd.to_numeric(S55['age5_[80, 85)'],errors='coerce').cumsum()
forplot['age85sum5']=pd.to_numeric(S55['age5_[85, 90)'],errors='coerce').cumsum()
forplot['age90sum5']=pd.to_numeric(S55['age5_[90, 125)'],errors='coerce').cumsum()

#forplot.subject_id

In [ ]:
##ages
bins60= [0,60,125]

forplot['AgeGroup60'] = pd.cut(forplot['ages'], bins=bins60,right=False)# labels=labels,
S60=pd.get_dummies(forplot.AgeGroup60, prefix='age60')

forplot['age1sum60']=pd.to_numeric(S60['age60_[0, 60)'], errors='coerce').cumsum()
forplot['age2sum60']=pd.to_numeric(S60['age60_[60, 125)'],errors='coerce').cumsum()

In [ ]:
print(forplot.shape)
len(forplot.subject_id.unique())

In [ ]:
##need to omit this until pull request is finished
## croms income

#bins_income= [0,20000,50000,100000, np.inf]
#labels_income = ['0-20k', '20k-50k', '50k-100k', '100k+']

#forplot['income']=pd.to_numeric(forplot.croms_income)
#forplot['IncomeGroup'] = pd.cut(forplot['income'], bins=bins_income,labels=labels_income,right=False)# labels=labels,
#forplot['IncomeGroup'] = forplot['IncomeGroup'].cat.add_categories('Don\'t know').fillna('Don\'t know')

#S6=pd.get_dummies(forplot.IncomeGroup, prefix='income')


In [ ]:
## education level
bins_edu = [0, 13, 15, 77, np.inf]
labels_edu = ['Below Highschool', 'Highschool or GED', 'Above Highschool or GED', 'Refused or do not know']

forplot['moca_edu']=pd.to_numeric(forplot.moca_edu)
forplot['EduGroup'] = pd.cut(forplot['moca_edu'], bins=bins_edu,labels=labels_edu,right=False)# labels=labels,
forplot['EduGroup'] = forplot['EduGroup'].cat.add_categories('Don\'t know').fillna('Don\'t know')
S6=pd.get_dummies(forplot.EduGroup, prefix='moca_edu')

## ALL participants accrued in AABC, by cohort (subjects, not visits)

In [ ]:
## Cohort x sex
pd.crosstab(forplot.Cohort,forplot.Sex,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#Cohort x Age
pd.crosstab(forplot.Cohort,forplot.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
## Cohort x race
pd.crosstab(forplot.Cohort,forplot.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
## Cohort x ethnicity
pd.crosstab(forplot.Cohort,forplot.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
# need to omit this until pull request is finished
#Cohort x Income
#pd.crosstab(forplot.Cohort,forplot.IncomeGroup,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#Cohort x Education
pd.crosstab(forplot.Cohort,forplot.EduGroup,margins=True)#.plot.bar(rot=45,title='AABC')



### AABC Female participants, by age and cohort

In [ ]:
#make a table by cohort, 5-year age
#FeMALES
females=forplot.loc[forplot.Sex=='Female'].copy()
pd.crosstab(females.Cohort,females.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

### AABC Male participants, by age and cohort 

In [ ]:
#make a table by cohort, 5-year age
males=forplot.loc[forplot.Sex=='Male'].copy()
pd.crosstab(males.Cohort,males.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

### AABC Female participants, by race and cohort

In [ ]:
#make a table by cohort, race
#FeMALES
females=forplot.loc[forplot.Sex=='Female'].copy()
pd.crosstab(females.Cohort,females.Race,margins=True)#.plot.bar(rot=45,title='AABC')

### AABC Male participants, by race and cohort 

In [ ]:
#make a table by cohort, race
#FeMALES
males=forplot.loc[forplot.Sex=='Male'].copy()
pd.crosstab(males.Cohort,males.Race,margins=True)#.plot.bar(rot=45,title='AABC')

### AABC Female participants, by ethnicity and cohort

In [ ]:
#make a table by cohort, ethnicity
#FeMALES
females=forplot.loc[forplot.Sex=='Female'].copy()
pd.crosstab(females.Cohort,females.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

### AABC Male participants, by ethnicity and cohort

In [ ]:
#make a table by cohort, ethnicity
#FeMALES
males=forplot.loc[forplot.Sex=='Male'].copy()
pd.crosstab(males.Cohort,males.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
## Project 3 crosstabs, Females 40-60 - move to separate report

In [ ]:
#Proj3=forplot[['Cohort','Sex','Race','Ethnicity','age','Site','AgeGroup5']].loc[(forplot.age.astype('float64')>=40) & (forplot.age.astype('float64')<=60) & (forplot.Sex=='Female')].copy()
#pd.crosstab(Proj3.Cohort,Proj3.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#pd.crosstab(Proj3.Cohort,Proj3.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#pd.crosstab(Proj3.Cohort,Proj3.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#pd.crosstab(Proj3.Site,Proj3.AgeGroup5,margins=True)

In [ ]:
#pd.crosstab(Proj3.Race,Proj3.AgeGroup5,margins=True)

## Accrued AABC site stats, by cohort

### MGH

In [ ]:
#make a table by cohort, sex
mgh=forplot.loc[forplot.Site=='MGH'].copy()
pd.crosstab(mgh.Cohort,mgh.Sex,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, 5-year age
#print("*******************")
#print("**** MGH ONLY *****")
mgh=forplot.loc[forplot.Site=='MGH'].copy()
pd.crosstab(mgh.Cohort,mgh.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, race
mgh=forplot.loc[forplot.Site=='MGH'].copy()
pd.crosstab(mgh.Cohort,mgh.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, ethnicity
mgh=forplot.loc[forplot.Site=='MGH'].copy()
pd.crosstab(mgh.Cohort,mgh.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

### UCLA

In [ ]:
#make a table by cohort, sex
ucla=forplot.loc[forplot.Site=='UCLA'].copy()
pd.crosstab(ucla.Cohort,ucla.Sex,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, 5-year age
#print("*******************")
#print("**** UCLA ONLY *****")
ucla=forplot.loc[forplot.Site=='UCLA'].copy()
pd.crosstab(ucla.Cohort,ucla.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, race
ucla=forplot.loc[forplot.Site=='UCLA'].copy()
pd.crosstab(ucla.Cohort,ucla.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, ethnicity
ucla=forplot.loc[forplot.Site=='UCLA'].copy()
pd.crosstab(ucla.Cohort,ucla.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

### WashU

In [ ]:
#make a table by cohort, sex
wu=forplot.loc[forplot.Site=='WashU'].copy()
pd.crosstab(wu.Cohort,wu.Sex,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#print("*******************")
#print("**** WU ONLY *****")
wu=forplot.loc[forplot.Site=='WashU'].copy()
pd.crosstab(wu.Cohort,wu.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, race
wu=forplot.loc[forplot.Site=='WashU'].copy()
pd.crosstab(wu.Cohort,wu.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, ethnicity
wu=forplot.loc[forplot.Site=='WashU'].copy()
pd.crosstab(wu.Cohort,wu.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

### UMN

In [ ]:
#make a table by cohort, Sex
umn=forplot.loc[forplot.Site=='UMN'].copy()
pd.crosstab(umn.Cohort,umn.Sex,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
umn=forplot.loc[forplot.Site=='UMN'].copy()
pd.crosstab(umn.Cohort,umn.AgeGroup5,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, race
umn=forplot.loc[forplot.Site=='UMN'].copy()
pd.crosstab(umn.Cohort,umn.Race,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
#make a table by cohort, ethnicity
umn=forplot.loc[forplot.Site=='UMN'].copy()
pd.crosstab(umn.Cohort,umn.Ethnicity,margins=True)#.plot.bar(rot=45,title='AABC')

## Section for date cutoff numbers, such as renewals, freezes etc. DO NOT USE...only accurate once


In [ ]:
## May 1st 2025 ACCRUED NUMBERS (not visits)
len(may2025.subject_id.unique())
forplot.shape
#forplot[['subject_id','redcap_event']].drop_duplicates()

In [ ]:

#forplot.loc[forplot.subject_id.isin(list(may2025.subject_id.unique()))][['Race','Ethnicity','Sex','ages','Cohort']].to_csv('./NIH_Participant_Level_Table_930.csv',index=False)
print(forplot.shape)
forplotmay2025=forplot.loc[forplot.subject_id.isin(list(may2025.subject_id))].copy()


In [ ]:
forplotmay2025['redcap_event']=''
forplotmay2025.loc[forplotmay2025.redcap_event_name.str.contains('v1_'),'redcap_event']='V1'
forplotmay2025.loc[forplotmay2025.redcap_event_name.str.contains('v2_'),'redcap_event']='V2'
forplotmay2025.loc[forplotmay2025.redcap_event_name.str.contains('v3_'),'redcap_event']='V3'
forplotmay2025.loc[forplotmay2025.redcap_event_name.str.contains('v4_'),'redcap_event']='V4'



In [ ]:
print("*******************")
print("**** Race x Sex in Non-Hispanics  *****")
nhisp=forplotmay2025.loc[forplotmay2025.Ethnicity=='Non-Hispanic'].copy()
pd.crosstab(nhisp.Race,nhisp.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
n=pd.crosstab(nhisp.Race,nhisp.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
n.to_csv('Non-Hispanic.csv')
n

In [ ]:
print("*******************")
print("**** Race x Sex in Hispanics  *****")
hisp=forplotmay2025.loc[forplotmay2025.Ethnicity=='Hispanic'].copy()
pd.crosstab(hisp.Race,hisp.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
h=pd.crosstab(hisp.Race,hisp.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
h.to_csv('Hispanic.csv')
h

In [ ]:
print("*******************")
print("**** Race x Sex for Unknown Ethnicities  *****")
unk=forplotmay2025.loc[forplotmay2025.Ethnicity=='Unknown'].copy()
pd.crosstab(unk.Race,unk.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
u=pd.crosstab(unk.Race,unk.Sex,margins=True)#.plot.bar(rot=45,title='AABC')
u.to_csv('Unknown.csv')
u

### End Section for Cutoff NUMBERS

## AABC Accrual PLOTS (subjects, not visits)


In [ ]:
#pd.crosstab(forplot.Counterbalance,forplot.Site,margins=True)#.plot.bar(rot=45,title='AABC')

In [ ]:
### create plot of AABC recruitment stats by SEX
# Create data
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.CB3sum) #[1,4,6,8,9]
y2=list(forplot.CB4sum)#[2,2,7,10,12]

# Basic stacked area chart.
#plt.stackplot(x,y1, y2, labels=['CB3:'+str(max(y1)),'CB4:'+str(max(y2))])
#plt.title("AABC")
#plt.legend(loc='upper left')
#plt.xlabel('Days Passed Since 1st Recruit')
#plt.ylabel('Number of Subjects');

In [ ]:
#PPP  


### create plot of AABC recruitment stats by SEX
# Create data
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.malesum) #[1,4,6,8,9]
y2=list(forplot.femalesum)#[2,2,7,10,12]

# Basic stacked area chart.
plt.stackplot(x,y1, y2, labels=['Male:'+str(max(y1)),'Female:'+str(max(y2))])
#plt.stackplot(x,y1, y2, labels=['Male: 346','Female: 468'])
plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');



In [ ]:
#By Age Bin
x=list(forplot.dayspassed) #range(1,6)
y2=list(forplot.age30sum)#[2,2,7,10,12]
y3=list(forplot.age40sum)
y4=list(forplot.age50sum)
y5=list(forplot.age60sum)
y6=list(forplot.age70sum)
y7=list(forplot.age80sum)
y8=list(forplot.age90sum)

# Basic stacked area chart.
plt.stackplot(x,y2,y3,y4,y5,y6,y7,y8, labels=['Age [30-40):'+str(max(y2)),'Age [40-50):'+str(max(y3)),'Age [50-60):'+str(max(y4)),'Age [60-70):'+str(max(y5)),'Age [70-80):'+str(max(y6)),'Age [80-90):'+str(max(y7)),'Age [90+):'+str(max(y8))])
plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');

In [ ]:
#By Age Bin
x=list(forplot.dayspassed) #range(1,6)
y2=list(forplot.age1sum60)#[2,2,7,10,12]
y3=list(forplot.age2sum60)


# Basic stacked area chart.
plt.stackplot(x,y2,y3, labels=['Age < 60: '+str(max(y2)), 'Age >= 60: '+str(max(y3))])
plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');

In [ ]:
#BY RACE
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.whitesum) #[1,4,6,8,9]
y2=list(forplot.blacksum)#[2,2,7,10,12]
y3=list(forplot.asiansum)
y4=list(forplot.moret1sum)
#y5=list(forplot.natpacsum)
y6=list(forplot['natamersum'])
y7=list(forplot['nasum'])



# Basic stacked area chart.
plt.stackplot(x,y1,y2,y3,y4,y6,y7,labels=['White:'+str(max(y1)),'Black:'+str(max(y2)),'Asian:'+str(max(y3)),'More than one Race:'+str(max(y4)),'Nat American/Alaskan:'+str(max(y6)),'Unknown/Unreported:'+str(max(y7))])
#plt.stackplot(x,y1,y2,y3,y4,y6,y7,labels=['White: 550','Black: 171','Asian: 50','More than one Race: 22','Nat American/Alaskan: 5','Unknown/Unreported: 15'])

plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');

In [ ]:
#BY ETHNICITY
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.nonhispanicsum) #[1,4,6,8,9]
y2=list(forplot.hispanicsum)#[2,2,7,10,12]
y3=list(forplot.unkhispsum)
# Basic stacked area chart.
plt.stackplot(x,y1, y2,y3, labels=['Non-Hispanic:'+str(max(y1)),'Hispanic:'+str(max(y2)),'Unknown or Not Reported:'+str(max(y3))])
#plt.stackplot(x,y1, y2,y3, labels=['Non-Hispanic: 734','Hispanic: 75','Unknown or Not Reported: 5'])
plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');

In [ ]:
#By Site
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.wusum) #[1,4,6,8,9]
y2=list(forplot.umnsum)#[2,2,7,10,12]
y3=list(forplot.mghsum)
y4=list(forplot.uclasum)
# Basic stacked area chart.
plt.stackplot(x,y1, y2,y3, y4,labels=['WU:'+str(max(y1)),'UMN:'+str(max(y2)),'MGH:'+str(max(y3)),'UCLA:'+str(max(y4))])
#plt.stackplot(x,y1, y2,y3, y4,labels=['WU: 282','UMN: 198','MGH: 180','UCLA: 154'])
plt.legend(loc='upper left')
plt.title("AABC")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects');


## AABC Crosstabulations

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Site).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.AgeGroup,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Site,margins=True)

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Race).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.AgeGroup,forplot.Race).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Race,margins=True)

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Ethnicity).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.AgeGroup,forplot.Ethnicity).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Ethnicity,margins=True)


In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Sex).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.AgeGroup,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Sex,margins=True)

In [ ]:
pd.crosstab(forplot.Sex,forplot.Site).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.Sex,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Sex,forplot.Site,margins=True)

In [ ]:
#Crosstabs x Site

pd.crosstab(forplot.Race,forplot.Site).plot.bar(rot=45,title='AABC')
#pd.crosstab(forplot.Race,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Race,forplot.Site,margins=True)

In [ ]:
pd.crosstab(forplot.Ethnicity,forplot.Site).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.Ethnicity,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Ethnicity,forplot.Site,margins=True)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Race,forplot.Sex).plot.bar(rot=45,title='AABC')
#pd.crosstab(forplot.Race,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Race,forplot.Sex,margins=True)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Ethnicity,forplot.Sex).plot.bar(rot=0,title='AABC')
#pd.crosstab(forplot.Ethnicity,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Ethnicity,forplot.Sex,margins=True)

## AABC only Visit Counts (not accrued subjects)

### Visit Counts during AABC

In [ ]:

x=aabcinvent.loc[(aabcinvent.passedscreen=='1') & (aabcinvent.redcap_event_name.str.contains('inperson')) & (aabcinvent.register_visit_complete=='2')].copy()
x =x.loc[~(x.subject_id=="HCA6539885")]
#print(x.shape)
#print(str(len(x.subject_id.unique())))
x['event_date'] = pd.to_datetime(x['event_date'])
x.columns
x['redcap_event']=''
x.loc[x.redcap_event_name.str.contains('v1_'),'redcap_event']='V1'
x.loc[x.redcap_event_name.str.contains('v2_'),'redcap_event']='V2'
x.loc[x.redcap_event_name.str.contains('v3_'),'redcap_event']='V3'
x.loc[x.redcap_event_name.str.contains('v4_'),'redcap_event']='V4'
#x.redcap_event.value_counts()

In [ ]:
#sites
S4=pd.get_dummies(x.site, prefix='site')
x['wusum']=pd.to_numeric(S4.site_4, errors='coerce').cumsum()
x['umnsum']=pd.to_numeric(S4.site_3, errors='coerce').cumsum()
x['mghsum']=pd.to_numeric(S4.site_1, errors='coerce').cumsum()
x['uclasum']=pd.to_numeric(S4.site_2, errors='coerce').cumsum()

xvar=list(x.dayspassed) #range(1,6)
y1=list(x.wusum) #[1,4,6,8,9]
y2=list(x.umnsum)#[2,2,7,10,12]
y3=list(x.mghsum)
y4=list(x.uclasum)
# Basic stacked area chart.
plt.stackplot(xvar,y1, y2,y3, y4,labels=['WU:'+str(max(y1)),'UMN:'+str(max(y2)),'MGH:'+str(max(y3)),'UCLA:'+str(max(y4))])
#plt.stackplot(x,y1, y2,y3, y4,labels=['WU: 282','UMN: 198','MGH: 180','UCLA: 154'])
plt.legend(loc='upper left')
plt.title("AABC: "+str(x.shape[0])+ " Visits in "+str(len(x.subject_id.unique()))+" subjects")
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Visits');

#x.dayspassed

In [ ]:
x['Site']=x.site.astype('str').replace({'1':'MGH','2':'UCLA','3':'UMN','4':'WashU'})
x['Race']=x.racial.replace({'1':'Nat Amer/Alaskan','2':'Asian','3':'Black','4':'Nat Hawaiian/PI','5':'White','6':'More than one','99':'Unknown'})
x['Ethnicity']=x.ethnic.replace({'1':'Hispanic','2':'Non-Hispanic','3':'Unknown'})
x['Sex']=x.sex.replace({'1':'Male','2':'Female'})

#forplot['Sex']=forplot.sex.replace({'1':'Male','2':'Female'})

x.groupby(['redcap_event', 'site']).size().reset_index(name='counts')
pd.crosstab(x.redcap_event,x.Site,margins=True)

In [ ]:
A=x.groupby(['subject_id']).count()#[['subject_id','study_id']]
B=pd.DataFrame(A).reset_index()[['subject_id','study_id']].rename(columns={'study_id':'number_visits'})
C=pd.DataFrame(B.groupby(['number_visits']).count()).reset_index().rename(columns={'subject_id':'number_subjects'})
C[['number_subjects','number_visits']].style.hide(axis="index")

In [ ]:
x.age_visit

In [ ]:
#basic demographics and ids/visits currently available 
x[['subject_id','redcap_event','site','age_visit','Sex','Race','Ethnicity']].head()
y=x[['subject_id','redcap_event','site','age_visit','Sex','Race','Ethnicity']].copy()
import numpy as np
import pandas as pd

y['age_visit'] = pd.to_numeric(y['age_visit'], errors='coerce')


# Round age_visit to 1 decimal place, only if it's a float
y['age_visit'] = y['age_visit'].apply(lambda x: round(x, 0) if isinstance(x, float) else x)

# Cap age_visit at 90
y['age_visit'] = y['age_visit'].apply(lambda x: 90 if isinstance(x, (int, float)) and x > 90 else x)

y['age_visit'] = y['age_visit'].astype('Int64')

In [ ]:
#y.to_csv('AABC_accrued_2025Jun4_basicdemog.csv',index=False)

### MRS visits 

In [ ]:
subjects7T=subjects7T.drop_duplicates()
print(subjects7T.shape)
MRS=pd.merge(x,subjects7T,left_on=['subject_id','redcap_event'],right_on=['subject','redcap_event'],how='outer',indicator=True)

MRS._merge.value_counts()
MRS=MRS.loc[MRS._merge=='both']
print("Subjects with MRS: Race x Sex")
#A=MRSMay2025.groupby(['subject']).count()#[['subject_id','study_id']]
#B=pd.DataFrame(A).reset_index()[['subject','redcap_event']].rename(columns={'redcap_event':'number_visits'})
#C=pd.DataFrame(B.groupby(['number_visits']).count()).reset_index().rename(columns={'subject':'number_subjects'})
#C[['number_subjects','number_visits']].style.hide(axis="index")
pd.crosstab(MRS.redcap_event,MRS.Site,margins=True)#.plot.bar(rot=45,title='AABC')


## MRS subjects

In [ ]:
MRSsubs=MRS.drop_duplicates(subset='subject_id')
pd.crosstab(MRSsubs.Race,MRSsubs.Sex,margins=True)#.plot.bar(rot=45,title='AABC')


In [ ]:
pd.crosstab(MRSsubs.Race,MRSsubs.Site,margins=True)#.plot.bar(rot=45,title='AABC')


## Visit Counts in the UNION of HCA and AABC

In [ ]:
idsV=ids.loc[ids.redcap_event.isin(['V1','V2'])].copy()
idsV.columns
#idsV[['subject','redcap_event','site','race','ethnic_group','M/F']]

In [ ]:
x.loc[x.sex=='2','M/F']='F'
x.loc[x.sex=='1','M/F']='M'
x['site']=x.site.astype('str').replace({'1':'MGH','2':'UCLA','3':'UMN','4':'WashU'})
x['race']=x.racial.replace({'1':'Nat Amer/Alaskan','2':'Asian','3':'Black','4':'Nat Hawaiian/PI','5':'White','6':'More than one','99':'Unknown'})
x['subject']=x.subject_id
#thnicity
x['ethnic_group']=x.ethnic.replace({'1':'Hispanic','2':'Non-Hispanic','3':'Unknown'})

#x[['subject','redcap_event','M/F','race','ethnic_group','site']]


In [ ]:
Union=pd.concat([idsV[['subject','redcap_event','M/F','race','ethnic_group','site']],x[['subject','redcap_event','M/F','race','ethnic_group','site']]])
Union.loc[Union.site=='UMinn','site']="UMN"
A=Union.groupby(['subject']).count()#[['subject_id','study_id']]
B=pd.DataFrame(A).reset_index()[['subject','redcap_event']].rename(columns={'redcap_event':'number_visits'})
C=pd.DataFrame(B.groupby(['number_visits']).count()).reset_index().rename(columns={'subject':'number_subjects'})
C[['number_subjects','number_visits']].style.hide(axis="index")


In [ ]:
#Union[['subject','redcap_event','M/F','race','ethnic_group','site']]


In [ ]:
#Union.groupby(['redcap_event', 'site']).size().reset_index(name='counts')
pd.crosstab(Union.redcap_event,Union.site,margins=True)

In [ ]:
#tasks based on current recruitment.  #define function which merges x with the aabc redcap data, using specified list of variables as input, and named output

In [ ]:
rid='66151'
keepcols=['study_id','redcap_event_name','bld_drawresults','bld_draw','bld_draw_nc','bld_serum','bld_serum_centrifuge','bld_serum_cryovials','bld_serum_miss']
outfile='MergeStatus_AMH_4June2025.csv'

aabcreport = redreport(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0],reportid=rid)
aabc=getframe(struct=aabcreport,api_url=config['Redcap']['api_url10'])
xspecial=x[['subject','subject_id','study_id','redcap_event','redcap_event_name','site','Site','M/F','sex','age','age_visit']].merge(aabc[keepcols],on=['study_id','redcap_event_name'],how='left')
xspecial.to_csv(outfile,index=False)

In [ ]:
#[j for j in x.columns if 'age' in j] 
#print(xspecial.shape)
#xspecial.head()